# Train Chrono and Highway GRU submissions
**Goal**: Train the newly created config for `gru_chrono_v1.yaml` and `gru_highway_v1.yaml` using 5 seeds (42-46) each to match the latest per-target ensemble logic.

**Recipe 1**: Vanilla GRU with Chrono Initialization
**Recipe 2**: Vanilla GRU with Highway Head

In [ ]:
# Cell 0: Mount Drive, download data from Kaggle
import os, json

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "FILL_IN"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

In [ ]:
# Cell 1: Setup — clone repo, link data
import os, subprocess
REPO = "/content/competition_package"

os.chdir("/content")
subprocess.run(["rm", "-rf", REPO], check=False)
subprocess.run(["git", "clone", "https://github.com/vincentvdo6/competition_package.git", REPO], check=True)
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

subprocess.run(["ln", "-sf", "/content/data/train.parquet", "datasets/train.parquet"], check=True)
subprocess.run(["ln", "-sf", "/content/data/valid.parquet", "datasets/valid.parquet"], check=True)

assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"
commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip()
print(f"Commit: {commit}")
print(f"GPU: {subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], text=True).strip()}")
print("Ready!")

In [ ]:
# Cell 2: Train Chrono Config
import os, subprocess, sys
os.chdir("/content/competition_package")

CONFIG = "configs/gru_chrono_v1.yaml"
SEEDS = list(range(42, 47))  # s42-s46

print(f"=== VANILLA GRU WITH CHRONO INITIALIZATION ===")
print(f"Config: {CONFIG}")
print(f"Seeds: {SEEDS[0]}-{SEEDS[-1]} ({len(SEEDS)} seeds)")
print("=" * 60, flush=True)

for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"Training gru_chrono_v1 seed {seed}")
    print(f"{'='*60}", flush=True)
    proc = subprocess.Popen(
        [sys.executable, "-u", "scripts/train.py",
         "--config", CONFIG,
         "--seed", str(seed), "--device", "cuda"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    for line in proc.stdout:
        print(line, end="", flush=True)
    proc.wait()
    if proc.returncode != 0:
        print(f"ERROR: seed {seed} failed with return code {proc.returncode}")

print(f"\nAll {len(SEEDS)} Chrono seeds done!")

In [ ]:
# Cell 3: Train Highway Config
import os, subprocess, sys
os.chdir("/content/competition_package")

CONFIG = "configs/gru_highway_v1.yaml"
SEEDS = list(range(42, 47))  # s42-s46

print(f"=== VANILLA GRU WITH HIGHWAY HEAD ===")
print(f"Config: {CONFIG}")
print(f"Seeds: {SEEDS[0]}-{SEEDS[-1]} ({len(SEEDS)} seeds)")
print("=" * 60, flush=True)

for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"Training gru_highway_v1 seed {seed}")
    print(f"{'='*60}", flush=True)
    proc = subprocess.Popen(
        [sys.executable, "-u", "scripts/train.py",
         "--config", CONFIG,
         "--seed", str(seed), "--device", "cuda"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    for line in proc.stdout:
        print(line, end="", flush=True)
    proc.wait()
    if proc.returncode != 0:
        print(f"ERROR: seed {seed} failed with return code {proc.returncode}")

print(f"\nAll {len(SEEDS)} Highway seeds done!")

In [ ]:
# Cell 4: Strip checkpoints + zip + save to Drive
import os, torch, glob, shutil
os.chdir("/content/competition_package")
os.makedirs("logs/slim", exist_ok=True)

for pt in sorted(glob.glob("logs/gru_*_v1_seed*.pt")):
    basename = os.path.basename(pt)
    if '_epoch' in basename:
        continue
    try:
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    except:
        continue
    slim = {
        "model_state_dict": ckpt["model_state_dict"],
        "config": ckpt.get("config", {}),
        "best_score": ckpt.get("best_score", None),
        "best_epoch": ckpt.get("best_epoch", None),
    }
    out = f"logs/slim/{basename}"
    torch.save(slim, out)
    orig = os.path.getsize(pt) / 1e6
    new = os.path.getsize(out) / 1e6
    print(f"{basename}: {orig:.1f}MB -> {new:.1f}MB")

shutil.make_archive("/content/chrono_highway_seeds", "zip",
                     "/content/competition_package/logs/slim")
sz = os.path.getsize("/content/chrono_highway_seeds.zip") / 1e6
print(f"\nchrono_highway_seeds.zip: {sz:.1f}MB")

shutil.copy("/content/chrono_highway_seeds.zip",
            "/content/drive/MyDrive/wunderfund/chrono_highway_seeds.zip")
print("Saved to Drive: MyDrive/wunderfund/chrono_highway_seeds.zip")